In [1]:
import Pkg; Pkg.add("RobotZoo"); Pkg.add("StaticArrays"); Pkg.add("PlotlyKaleido")
Pkg.activate(@__DIR__)
Pkg.instantiate()
import Pkg;
import MathOptInterface as MOI
import Ipopt 
import FiniteDiff
import ForwardDiff as FD
import Convex as cvx 
import ECOS
using LinearAlgebra
using Plots; plotly()
using Random
using JLD2
using Test
using MeshCat
const mc = MeshCat
# using TrajOptPlots
# using StaticArrays
using Printf
using RobotZoo
using StaticArrays

    Updating registry at `C:\Users\ologa\.julia\registries\General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
   Installed xkbcommon_jll ─ v1.4.1+1
   Installed Wayland_jll ─── v1.21.0+1
   Installed EpollShim_jll ─ v0.0.20230411+0
  No Changes to `C:\Users\ologa\Documents\CMU\Semester2\16.745-OptimalControlReinforcementLearning\MonkeyBarBot\MonkeyBarBot\Project.toml`
    Updating `C:\Users\ologa\Documents\CMU\Semester2\16.745-OptimalControlReinforcementLearning\MonkeyBarBot\MonkeyBarBot\Manifest.toml`
  [2702e6a9] + EpollShim_jll v0.0.20230411+0
  [a2964d1f] ↑ Wayland_jll v1.21.0+0 ⇒ v1.21.0+1
  [d8fb68d0] ↑ xkbcommon_jll v1.4.1+0 ⇒ v1.4.1+1
Precompiling project...
  ✓ EpollShim_jll
  ✓ Wayland_jll
  ✓ xkbcommon_jll
  ✓ Qt5Base_jll
  ✓ GR_jll
  ✓ GR
  ✓ Plots
  7 dependencies successfully precompiled in 88 seconds (202 already precompiled)
   Resolving package versions...
  No Changes to `C:\Users\ologa\Documents\CMU\Semest

In [9]:
include(joinpath(@__DIR__, "utils","fmincon.jl"))
include(joinpath(@__DIR__, "utils", "monkeyBotDynamics.jl"))

monkeybot_dynamics (generic function with 2 methods)

In [10]:
#Copied from RobotZoo.acrobot() src
function monkeybot_dynamics(params::NamedTuple, x::Vector, u::Vector)
    g = 9.81
    m1,m2 = params.m1, params.m2
    l1,l2 = params.l1, params.l2
    J1,J2 = params.I1, params.I2
    θ1,    θ2    = x[1], x[2]
    θ1dot, θ2dot = x[3], x[4]
    s1,c1 = sincos(θ1)
    s2,c2 = sincos(θ2)
    c12 = cos(θ1 + θ2)

    # mass matrix
    m11 = m1*l1^2 + J1 + m2*(l1^2 + l2^2 + 2*l1*l2*c2) + J2
    m12 = m2*(l2^2 + l1*l2*c2) + J2
    m22 = l2^2*m2 + J2
    M = [m11 m12; m12 m22]

    # bias term
    tmp = l1*l2*m2*s2
    b1 = -(2 * θ1dot * θ2dot + θ2dot^2)*tmp
    b2 = tmp * θ1dot^2
    B = [b1, b2]

    # friction
#     c = 1.0
#     C = [c*θ1dot, c*θ2dot]

    # gravity term
    g1 = ((m1 + m2)*l2*c1 + m2*l2*c12) * g
    g2 = m2*l2*c12*g
    G = [g1, g2]
    # equations of motion
    τ = [0, u[1]]
    θddot = M\(τ - B - G)
    return [θ1dot, θ2dot, θddot[1], θddot[2]]
end

#Jump maps
function jump1_map(x)
    #Hand 1 experiences inelastic collision with bar
    
    #Zero out hand 1 linear velocity
    #Constraint is that transform to hand 1 velocity equals 0 and hand 1 position equals bar position
    θ1, θ2 = x[1], x[2]
    θ1_star = -(pi + θ1)
    θ2_star = -θ2
    
    return [θ1_star, θ2_star, 0, 0]
end

#Generate Integrator Function
function dynamics_rk4(params::NamedTuple, ode::Function, x::Vector, u::Vector, dt::Real)::Vector
    k1 = dt * ode(params, x,        u)
    k2 = dt * ode(params, x + k1/2, u)
    k3 = dt * ode(params, x + k2/2, u)
    k4 = dt * ode(params, x + k3,   u)
    return x + (1/6)*(k1 + 2*k2 + 2*k3 + k4)
end


dynamics_rk4 (generic function with 1 method)

In [15]:
#Cost Functions
function cost(params::NamedTuple, Z::Vector)::Real
    #Calculate cost over whole trajectory
    N, xic, idx = params.N, params.xic, params.idx
    Q, R, Qf = params.Q, params.R, params.Qf
    
    cost = 0.0
    for k = 1:(N-1)
        ##Maybe try using Xref for shoulder trajectory as cost to make sure it stays on that path
            ## Might interfere with free swing at maximum swing
        xk = Z[idx.x[k]]
        uk = Z[idx.u[k]]
        cost += 0.5*(xk'*Qf*xk + uk'*R*uk)
    end
    xN = Z[idx.x[N]]
    cost += 0.5*(xN'*Qf*xN)
    
    return cost
end

function cost_freeTime(params::NamedTuple, Z::Vector)::Real
    #Calculate cost over whole trajectory
    N, idx = params.N, params.idx
    Q, R, Qf = params.Q, params.R, params.Qf
    dt = params.dt
    
    cost = 0.0
    for k = 1:(N-1)
        ##Maybe try using Xref for shoulder trajectory as cost to make sure it stays on that path
            ## Might interfere with free swing at maximum swing
        xk = Z[idx.x[k]]
        uk = Z[idx.u[k]]
        uk_tilde = [0; dt]-uk
        cost += 0.5*(xk'*Qf*xk + uk_tilde'*R*uk_tilde)
    end
    xN = Z[idx.x[N]]
    cost += 0.5*(xN'*Qf*xN)
    
    return cost
end


cost_freeTime (generic function with 1 method)

In [16]:
#Generate initial trajectory
function generateInitialTrajectory(params::NamedTuple, Z::Vector)
end

function generateSimpleTrajectory(params::NamedTuple, Z::Vector)
    #Generate initial simplified version of center body trajectory (sinusoidal path)
    
end

generateSimpleTrajectory (generic function with 1 method)

In [17]:
function create_idx(nx,nu,N) #copied from homeworks
    # create idx for indexing convenience
    # x_i = Z[idx.x[i]]
    # u_i = Z[idx.u[i]]
    # h_i = Z[idx.h[i]]
    # and stacked dynamics constraints of size nx are 
    # c[idx.c[i]] = <dynamics constraint at time step i>
    #
    # feel free to use/not use this 
    
    # our Z vector is [x0, u0, x1, u1, …, xN]
    nz = (N-1) * nu + N * nx # length of Z 
    x = [(i - 1) * (nx + nu) .+ (1 : nx) for i = 1:N]
    u = [(i - 1) * (nx + nu) .+ ((nx + 1):(nx + nu)) for i = 1:(N - 1)]
    
    # constraint indexing for the (N-1) dynamics constraints when stacked up
    c = [(i - 1) * (nx) .+ (1 : nx) for i = 1:(N - 1)]
    nc = (N - 1) * nx # (N-1)*nx 
    
#     return (nx=nx, nu=nu, N=N, nz=nz, nc=nc, x=x, u=u, h=h, c = c)
    return (nx=nx, nu=nu, N=N, nz=nz, nc=nc, x=x, u=u, c = c)
end

function dynamics_constraints(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    M1, J1 = params.M1, params.J1
    
    c = zeros(eltype(Z), idx.nc)
    
    for k = 1:(N-1)
        xk = Z[idx.x[k]]
        uk = Z[idx.u[k]]
        if !(k in J1) #Stance 1 dynamics
            xk1 = dynamics_rk4(params, monkeybot_dynamics, xk, uk, dt)
            c[idx.c[k]] = Z[idx.x[k+1]] - xk1 #Add dynamics constraint on state
        else
            xk1 = jump1_map(dynamics_rk4(params, monkeybot_dynamics, xk, uk, dt))
            c[idx.c[k]] = Z[idx.x[k+1]] - xk1 #Add dynamics constraint on state
        end
    end
    
    return c 
end

function dynamics_constraints_freeTime(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    M1, J1 = params.M1, params.J1
    
    c = zeros(eltype(Z), idx.nc)
    
    for k = 1:(N-1)
        xk = Z[idx.x[k]]
        ui = Z[idx.u[k]]
        uk = [ui[1]]
        hk = ui[2]
        if !(k in J1) #Stance 1 dynamics
            xk1 = dynamics_rk4(params, monkeybot_dynamics, xk, uk, hk)
            c[idx.c[k]] = Z[idx.x[k+1]] - xk1 #Add dynamics constraint on state
        else
            xk1 = jump1_map(dynamics_rk4(params, monkeybot_dynamics, xk, uk, hk))
            c[idx.c[k]] = Z[idx.x[k+1]] - xk1 #Add dynamics constraint on state
        end
    end
    
    return c 
end

function equality_constraint(params::NamedTuple, Z::Vector)::Vector
    N, idx, xic, xg = params.N, params.idx, params.xic, params.xg 
    l1, l2 = params.l1, params.l2
    BarCoords = params.BarCoords
    J1 = params.J1
    
    #Stack up all equality constraints
    initialCondition = Z[idx.x[1]] - xic
    terminalCondition = Z[idx.x[1]] - xg
    dynamicsConstraints = dynamics_constraints(params, Z)
    
    return [initialCondition; terminalCondition; dynamicsConstraints]
end

function equality_constraint_freeTime(params::NamedTuple, Z::Vector)::Vector
    N, idx, xic, xg = params.N, params.idx, params.xic, params.xg 
    l1, l2 = params.l1, params.l2
    BarCoords = params.BarCoords
    J1 = params.J1
    
    #Stack up all equality constraints
    initialCondition = Z[idx.x[1]] - xic
    terminalCondition = Z[idx.x[N]] - xg
    dynamicsConstraints = dynamics_constraints_freeTime(params, Z)
    
    return [initialCondition; terminalCondition; dynamicsConstraints]
end

function inequality_constraint(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
        
#     #Write inequality constraints for the problem
#     c = zeros(eltype(Z), 3*N
#     for k = 1:(N-1)
# #         uk = Z[idx.u[k]]
# #         c[3*k] = uk[2]
#         c[k] = xk[1]
#     end
    return []
end


inequality_constraint (generic function with 1 method)

In [13]:
function calculate_monkey_bar_trajectory(m1, m2, BarSeparation, B_N) 
    # problem size 
    nx = 4
    nu = 2
    B_1 = [i for i = 1:length(B_N)]
    NBars = length(B_1)+1
#     B_N = [30,20,20,20,20]
    dt = 0.05
    g = 9.81
    l1 = 1
    l2 = 1
#     m1 = 1
#     m2 = 1
    I1 = m1*0.2
    I2 = m1*0.2

#     BarSeparation = [1.5, 1.5, 1.5, 1.5, 1.5]

    BarCoords = [zeros(2) for i = 1:(NBars+1)]

    for i = 2:(NBars)
        barX = BarCoords[i-1][1] + BarSeparation[i-1]
        barY = 0
        BarCoords[i] = [barX; barY]
    end
    BarCoords

    X = [zeros(nx) for i = 1:B_N[1]]
    U = [zeros(nu) for i = 1:(B_N[1]-1)]

    barDist = abs(BarCoords[2][1] - BarCoords[1][1])

    θ1g = -acos((barDist^2 + 1^1 - 1^2)/(2*barDist*1))
    θ2g = pi - acos((1^2 + 1^2 - barDist^2)/(2*1*1))
    xic = [-pi/2,0,0,0] #Start hanging
    xg = [θ1g,θ2g,0,0]

    # index sets 
    M1 = [i for i = 1:length(B_N[1])]
    J1 = B_N[1]

    # LQR cost function
    Q = diagm([10; 10; 10; 10]);
    R = diagm([0.1; 1]); #Make smaller
    Qf = 10*Q;

    # create indexing utilities 
    idx = create_idx(nx,nu,B_N[1])
    # Params
    params = (
        nx = nx,
        nu = nu,
        dt = dt, 
        N = B_N[1], 
        M1 = M1, 
        J1 = J1, 
        BarCoords = BarCoords,
        xic = xic, 
        xg = xg,
        idx = idx,
        Q = Q, R = R, Qf = Qf,
        g = g,
        l1 = l1, l2 = l2,
        I1 = I1, I2 = I2,
        m1 = m1, m2 = m2,
    )

    #Primal bounds
    x_l = zeros(idx.nz)
    x_u =  zeros(idx.nz)

    for i = 1:B_N[1]
        x_l[idx.x[i]] = [-Inf, -Inf, -Inf, -Inf]
        x_u[idx.x[i]] = [Inf, Inf, Inf, Inf]
        if (i < B_N[1])
            x_l[idx.u[i]] = [-Inf, dt/2]
            x_u[idx.u[i]] = [Inf, 2*dt]
        end
    end

    #Inequality constraint bounds
    c_l = []
    c_u = []

    #Initialize z0 with reference shoulder trajectory
    z0 = zeros(idx.nz) # update this

    # adding a little noise to the initial guess is a good idea 
    z0 = z0 + (1e-6)*randn(idx.nz)

    #Choose diff type
    diff_type = :auto 

    #Solve trajectory
    Z = fmincon(cost_freeTime,equality_constraint_freeTime,inequality_constraint,
                x_l,x_u,c_l,c_u,z0,params,diff_type;
                tol = 1e-6, c_tol = 1e-6, max_iters = 10_000, verbose = false)

    X[1:B_N[1]] = [Z[idx.x[i]] for i = 1:B_N[1]]
    U[1:(B_N[1]-1)] = [Z[idx.u[i]] for i = 1:(B_N[1]-1)]
    dt_vec = [U[i][2] for i = 1:(B_N[1]-1)]
    tvec = vcat([0;[sum(dt_vec[1:i]) for i = 1:length(dt_vec)]])

    for barIter = 2:(NBars-1)
        barDist = abs(BarCoords[barIter+1][1] - BarCoords[barIter][1])
        θ1g = -acos((barDist^2 + 1^1 - 1^2)/(2*barDist*1))
        θ2g = pi - acos((1^2 + 1^2 - barDist^2)/(2*1*1))
    #     @show sum(B_N[1:barIter-1])
        xic = jump1_map(X[sum(B_N[1:barIter-1])]) #Start hanging
        xg = [θ1g,θ2g,0,0]

        idx = create_idx(nx,nu,B_N[barIter])

        params = (
            nx = nx,
            nu = nu,
            dt = dt, 
            N = B_N[barIter], 
            M1 = [i for i = 1:B_N[barIter]], 
            J1 = B_N[barIter], 
            BarCoords = BarCoords,
            xic = xic, 
            xg = xg,
            idx = idx,
            Q = Q, R = R, Qf = Qf,
            g = g,
            l1 = l1, l2 = l2,
            I1 = I1, I2 = I2,
            m1 = m1, m2 = m2,
        )

        #Primal bounds
        x_l = zeros(idx.nz)
        x_u =  zeros(idx.nz)

        # Minimum Time bounds
        for i = 1:B_N[barIter]
            x_l[idx.x[i]] = [-Inf, -Inf, -Inf, -Inf]
            x_u[idx.x[i]] = [Inf, Inf, Inf, Inf]
            if (i < B_N[barIter])
                x_l[idx.u[i]] = [-Inf, dt/2]
                x_u[idx.u[i]] = [Inf, 2*dt]
            end
        end

        #Inequality constraint bounds
        c_l = []
        c_u = []

        #Initialize z0
        z0 = zeros(idx.nz) # update this

        # adding a little noise to the initial guess is a good idea 
        z0 = z0 + (1e-6)*randn(idx.nz)

        #Choose diff type
        diff_type = :auto 

        #Solve trajectory
        Z = fmincon(cost_freeTime,equality_constraint_freeTime,inequality_constraint,
                    x_l,x_u,c_l,c_u,z0,params,diff_type;
                    tol = 1e-6, c_tol = 1e-6, max_iters = 10_000, verbose = false)

        X = vcat([X; [Z[idx.x[i]] for i = 1:B_N[barIter]]])
        U = vcat([U; [Z[idx.u[i]] for i = 1:(B_N[barIter]-1)]])
        dt_vec = [U[i][2] for i = 1:(B_N[barIter]-1)]
        tvec_curr = vcat([(tvec[end]+dt); [tvec[end]+dt+sum(dt_vec[1:i]) for i = 1:length(dt_vec)]])
        tvec = vcat([tvec;tvec_curr])

    end
    return X, U, tvec
end

calculate_monkey_bar_trajectory (generic function with 1 method)

In [14]:
B_N = [30,20,20,20,20]
B_1 = [i for i = 1:length(B_N)]
BarSeparation = [1.5, 1.5, 1.5, 1.5, 1.5]
dt = 0.05
m1 = 1
m2 = 1

#Change Mass Tests
# X1, U1, tvec1 = calculate_monkey_bar_trajectory(1, 1, BarSeparation, B_N) 
# X2, U2, tvec2 = calculate_monkey_bar_trajectory(3, 1, BarSeparation, B_N) 
# X3, U3, tvec3 = calculate_monkey_bar_trajectory(5, 1, BarSeparation, B_N) 
# X4, U4, tvec4 = calculate_monkey_bar_trajectory(1, 2, BarSeparation, B_N) 

#Change Bar Separation Tests
BarSeparation1 = [1.3, 1.3, 1.3, 1.3, 1.3]
BarSeparation2 = [1.5, 1.5, 1.5, 1.5, 1.5]
BarSeparation3 = [1.8, 1.8, 1.8, 1.8, 1.8]
BarSeparation4 = [1.3, 1.5, 1.1, 1.9, 1.4]
X1, U1, tvec1 = calculate_monkey_bar_trajectory(m1, m1, BarSeparation1, B_N) 
X2, U2, tvec2 = calculate_monkey_bar_trajectory(m1, m2, BarSeparation2, B_N) 
# X3, U3, tvec3 = calculate_monkey_bar_trajectory(m1, m2, BarSeparation3, B_N) 
# X4, U4, tvec4 = calculate_monkey_bar_trajectory(m1, m2, BarSeparation4, B_N) 


LoadError: UndefVarError: cost_freeTime not defined

In [40]:
@show maximum([X2[i][3] for i = 1:size(X1,1)])
@show minimum([X2[i][3] for i = 1:size(X1,1)])
@show maximum([X2[i][4] for i = 1:size(X1,1)])
@show minimum([X2[i][4] for i = 1:size(X1,1)])

maximum([(X2[i])[3] for i = 1:size(X1, 1)]) = 3.5530018421322493
minimum([(X2[i])[3] for i = 1:size(X1, 1)]) = -0.7744455895624399
maximum([(X2[i])[4] for i = 1:size(X1, 1)]) = 3.951644182397878
minimum([(X2[i])[4] for i = 1:size(X1, 1)]) = -8.85695022195887


-8.85695022195887

In [41]:
#Plot joint angles
# fig = plot(tvec1, [X1[i][1] for i = 1:size(X1,1)], label = "m1:m2 = 1:1",
#             title = "Theta 1 v.s. Time", xlabel = "Time [s]", ylabel = "Angle [rad]")
# plot!(tvec2, [X2[i][1] for i = 1:size(X2,1)], label = "m1:m2 = 3:1")
# plot!(tvec3, [X3[i][1] for i = 1:size(X3,1)], label = "m1:m2 = 5:1")
# display(plot!(tvec4, [X4[i][1] for i = 1:size(X4,1)], label = "m1:m2 = 1:2"))
# savefig("Change_Mass_Results_Theta1.png")

# fig = plot(tvec1, [X1[i][2] for i = 1:size(X1,1)], label = "m1:m2 = 1:1",
#             title = "Theta 2 v.s. Time", xlabel = "Time [s]", ylabel = "Angle [rad]")
# plot!(tvec2, [X2[i][2] for i = 1:size(X2,1)], label = "m1:m2 = 3:1")
# plot!(tvec3, [X3[i][2] for i = 1:size(X3,1)], label = "m1:m2 = 5:1")
# display(plot!(tvec4, [X4[i][2] for i = 1:size(X4,1)], label = "m1:m2 = 1:2"))
# savefig("Change_Mass_Results_Theta2.png")

# fig = plot(tvec1, [X1[i][1] for i = 1:size(X1,1)], label = "Test 1",
#             title = "Theta 1 v.s. Time", xlabel = "Time [s]", ylabel = "Angle [rad]")
# plot!(tvec2, [X2[i][1] for i = 1:size(X2,1)], label = "Test 2")
# display(plot!(tvec3, [X3[i][1] for i = 1:size(X3,1)], label = "Test 3"))
# # display(plot!(tvec4, [X4[i][1] for i = 1:size(X4,1)], label = "Test 4"))
# savefig("Change_Bar_Length_Results_Theta1.png")

# fig = plot(tvec1, [X1[i][2] for i = 1:size(X1,1)], label = "Test 1",
#             title = "Theta 2 v.s. Time", xlabel = "Time [s]", ylabel = "Angle [rad]")
# plot!(tvec2, [X2[i][2] for i = 1:size(X2,1)], label = "Test 2")
# display(plot!(tvec3, [X3[i][2] for i = 1:size(X3,1)], label = "Test 3"))
# # display(plot!(tvec4, [X4[i][2] for i = 1:size(X4,1)], label = "Test 4"))
# savefig("Change_Bar_Length_Results_Theta2.png")

# fig = plot(tvec1, [X1[i][1] for i = 1:size(X1,1)], label = "m1:m2 = 1:1", linecolor = "red",
#             title = "Theta v.s. Time", xlabel = "Time [s]", ylabel = "Angle [rad]")
# plot!(tvec2, [X2[i][1] for i = 1:size(X2,1)], label = "m1:m2 = 3:1", linecolor = "blue")
# plot!(tvec3, [X3[i][1] for i = 1:size(X3,1)], label = "m1:m2 = 5:1", linecolor = "green")
# plot!(tvec4, [X4[i][1] for i = 1:size(X4,1)], label = "m1:m2 = 1:2", linecolor = "purple")
# plot!(tvec1, [X2[i][2] for i = 1:size(X2,1)], label = "m1:m2 = 1:1", linecolor = "red")
# plot!(tvec2, [X2[i][2] for i = 1:size(X2,1)], label = "m1:m2 = 3:1", linecolor = "blue")
# plot!(tvec3, [X3[i][2] for i = 1:size(X3,1)], label = "m1:m2 = 5:1", linecolor = "green")
# display(plot!(tvec4, [X4[i][2] for i = 1:size(X4,1)], label = "m1:m2 = 1:2", linecolor = "purple"))
# savefig("Change_Mass_Results_Theta.png")

# fig = plot(tvec1, [U1[i][1] for i = 1:size(U1,1)], label = "Test 1", linecolor = "red",
#             title = "Torque v.s. Time", xlabel = "Time [s]", ylabel = "Torque [Nm]")
# plot!(tvec2, [U2[i][1] for i = 1:size(U2,1)], label = "Test 2", linecolor = "blue")
# plot!(tvec3, [U3[i][1] for i = 1:size(U3,1)], label = "Test 3", linecolor = "green")
# display(plot!(tvec4, [U4[i][1] for i = 1:size(U4,1)], label = "Test 4", linecolor = "purple"))
# savefig("Change_Mass_Results_Torque.png")



LoadError: UndefVarError: X3 not defined

In [49]:
#Post process to get x and y of hands
# @show dt_vec1 = [U[i][2] for i = 1:(B_N[1]-1)]
# @show dt_vec2 = [U[i][2] for i = B_N[1]:(B_N[2]-1)]
# @show dt_vec3 = [U[i][2] for i = (B_N[2]-1):(B_N[3]-1)]
# @show dt_vec4 = [U[i][2] for i = (B_N[2]-2):(B_N[4]-1)]
# @show dt_vec5 = [U[i][2] for i = (B_N[2]-3):(B_N[5]-1)]
# u = [U[i][1] for i = 1:size(U,1)]
bar = zeros(sum(B_N[:]))
xB = zeros(sum(B_N[:]))
yB = zeros(sum(B_N[:]))
x1 = zeros(sum(B_N[:]))
y1 = zeros(sum(B_N[:]))
x2 = zeros(sum(B_N[:]))
y2 = zeros(sum(B_N[:]))
NBars = length(B_N)+1
BarSeparation = 
BarCoords = [zeros(2) for i = 1:(NBars+1)]
for i = 2:(NBars)
    barX = BarCoords[i-1][1] + BarSeparation2[i-1]
    barY = 0
    BarCoords[i] = [barX; barY]
end
l1 = 1
l2 = 1
barIdx = 1
for k = 1:sum(B_N)
    if (k > sum(B_N[1:barIdx]))
        barIdx += 1
    end
    bar[k] = barIdx
    xB[k] = BarCoords[B_1[barIdx]][1]
    yB[k] = BarCoords[B_1[barIdx]][2]
    θ1, θ2, θ1dot, θ2dot = X2[k]
    x1[k] = xB[k] + l1*cos(θ1)
    y1[k] = yB[k] + l1*sin(θ1)
    x2[k] = x1[k] + l2*cos(θ1 + θ2)
    y2[k] = y1[k] + l2*sin(θ1 + θ2)
end


a = @animate for i ∈ 1:length(x1)-1
    h = scatter([BarCoords[i][1] for i = 1:NBars], [BarCoords[i][2] for i = 1:NBars], marker = :circle, seriescolor = :blue, label = "bar",
                    aspect_ratio = :equal, xlims = (-2,8), ylims = (-4,4))
    plot!([tuple([xB[i]; x1[i]], [yB[i]; y1[i]])], marker = :circle, seriescolor = :red, legend = false)
    plot!([tuple([x1[i]; x2[i]], [y1[i]; y2[i]])], marker = :circle, seriescolor = :green, legend = false)
#     Plots.frame(a, b)
end
gif(a, fps = 4, "Bar-Separation-Test-1.gif")

h = scatter([BarCoords[i][1] for i = 1:NBars], [BarCoords[i][2] for i = 1:NBars], marker = :circle, seriescolor = :blue, label = "bar",
aspect_ratio = :equal, xlims = (-2,8), ylims = (-4,4))
plot!(x1, y1, marker = :circle, seriescolor = :red, legend = false)
display(plot!(x2, y2, marker = :circle, seriescolor = :green, legend = false))

[ Info: Saved animation to C:\Users\ologa\Documents\CMU\Semester2\16.745-OptimalControlReinforcementLearning\Optimal-Controls-Gymnastics-Bot\MonkeyBarBot\Bar-Separation-Test-1.gif


In [31]:
traj_All = Array{Any}(undef, length(B_N))

tvec_animation = [sum(tvec2[1:i]) for i = 1:length(U2[:][2])]
bar_animation = zeros(sum(B_N[:]))
xB_animation = zeros(sum(B_N[:]))
yB_animation = zeros(sum(B_N[:]))
x1_animation = zeros(sum(B_N[:]))
y1_animation = zeros(sum(B_N[:]))
x2_animation = zeros(sum(B_N[:]))
y2_animation = zeros(sum(B_N[:]))
θ1_animation = zeros(sum(B_N[:]))
θ2_animation = zeros(sum(B_N[:]))
@show 1

                                    
idx = 0
for i = 1:length(B_N)
    @show tvec[sum(B_N[1:i])]
    @show 0:dt:tvec2[sum(B_N[1:i])]
    @show [tvec_animation[idx]+dt*(j) for j = 0:tvec2[sum[1:B_N[i]]]]
    @show tvec_animation[idx+1:B_N[i]] = 0:dt:tvec2[sum(B_N[1:i])] .+ tvec2
#     for k = 1:B_N[i]
#         currTime = tvec_animation[k+idx])]
#     end
    idx = B_N[i]
end
    
include(joinpath(@__DIR__, "utils","walker.jl"))
model = (g = 9.81, mb= 0.0, mf = 1.0, ℓ_min = 0.5, ℓ_max = 1.5,l1=l1,l2 = l2)
vis = Visualizer()
dt = params.dt
build_walker!(vis, model::NamedTuple)
anim = mc.Animation(floor(Int,tvec[end]/dt))
X_all = X2
bar_x_all = BarCoords[1][1]*ones(length(X2))
bar_y_all = BarCoords[1][2]*ones(length(X2))
contact_modes_all = ones(length(X2))


# for j = 2:length(BarCoords)
#     contact_mode_j = j%2
#     X_j = X2
#     X_all = vcat(X_all,X_j)
#     contact_modes_all = vcat(contact_modes_all,contact_mode_j*ones(length(X_j)))
#     bar_x_all = vcat(bar_x_all,BarCoords[j][1]*ones(length(X_j)))
#     bar_y_all = vcat(bar_y_all,BarCoords[j][2]*ones(length(X_j)))
# end



1 = 1


┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
└ http://127.0.0.1:8704


110-element Vector{Float64}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 ⋮
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

In [53]:
nx = 4
nu = 2
B_1 = [i for i = 1:length(B_N)]
NBars = length(B_1)+1
#     B_N = [30,20,20,20,20]
dt = 0.05
g = 9.81
l1 = 1
l2 = 1
#     m1 = 1
#     m2 = 1
I1 = m1*0.2
I2 = m1*0.2
BarSeparation = [1.5, 1.5, 1.5, 1.5, 1.5]

BarCoords = [zeros(2) for i = 1:(NBars+1)]

for i = 2:(NBars)
    barX = BarCoords[i-1][1] + BarSeparation[i-1]
    barY = 0
    BarCoords[i] = [barX; barY]
end
BarCoords

X = [zeros(nx) for i = 1:B_N[1]]
U = [zeros(nu) for i = 1:(B_N[1]-1)]

barDist = abs(BarCoords[2][1] - BarCoords[1][1])

θ1g = -acos((barDist^2 + 1^1 - 1^2)/(2*barDist*1))
θ2g = pi - acos((1^2 + 1^2 - barDist^2)/(2*1*1))
xic = [-pi/2,0,0,0] #Start hanging
xg = [θ1g,θ2g,0,0]
params = (
            dt = dt, 
            BarCoords = BarCoords,
            xic = xic, 
            xg = xg,
            idx = idx,
            g = g,
            l1 = l1, l2 = l2,
            I1 = I1, I2 = I2,
            m1 = m1, m2 = m2,
        )
for i = 1:sum(B_N)
#     xB = bar_x_all[i]
#     yB = bar_y_all[i]
    θ1, θ2, θ1dot, θ2dot = X2[i]
#     θ1 = θ1_guess[i]
#     θ2 = θ2_guess[i]
    state_i = [θ1,θ2,xB[i],yB[i]]
    contact_mode = bar[i]%2
    mc.atframe(anim, i) do
        update_monkey_pose!(vis,contact_mode,model::NamedTuple,params::NamedTuple, state_i)
    end
end

mc.setanimation!(vis, anim)
display(render(vis))

MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), UInt8[0x83, 0xa7, 0x6f, 0x70, 0x74, 0x69, 0x6f, 0x6e, 0x73, 0x82  …  0x5f, 0x61, 0x6e, 0x69, 0x6d, 0x61, 0x74, 0x69, 0x6f, 0x6e], Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("robot" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("Rfoot" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x52, 0x66, 0x6f, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x52, 0x66, 0x6f, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "torso" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("body" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x74, 0x6f, 0x72, 0x73, 0x6f, 0x2f, 0x62, 0x6f, 0x64, 0x79], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "Laxle" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x6f, 0x72, 0x73, 0x6f, 0x2f, 0x4c, 0x61, 0x78, 0x6c, 0x65], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6f, 0x72, 0x73, 0x6f, 0x2f, 0x4c, 0x61, 0x78, 0x6c, 0x65], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "Lleg" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x4c, 0x6c, 0x65, 0x67, 0x2f, 0x67, 0x65, 0x6f, 0x6d], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "Rleg" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x52, 0x6c, 0x65, 0x67, 0x2f, 0x67, 0x65, 0x6f, 0x6d], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "Raxle" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x6f, 0x72, 0x73, 0x6f, 0x2f, 0x52, 0x61, 0x78, 0x6c, 0x65], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6f, 0x72, 0x73, 0x6f, 0x2f, 0x52, 0x61, 0x78, 0x6c, 0x65], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "Lfoot" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x4c, 0x66, 0x6f, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x4c, 0x66, 0x6f, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))))))), Set{Any}(), ip"127.0.0.1", 8704))